# RAGAS 를 활용한 평가

**참고**
- RAGAS: https://docs.ragas.io/en/stable/concepts/metrics/overview/

아래의 주석을 해제한 후 실행하여 패키지를 설치 후 진행해주세요

In [25]:
%pip install -qU faiss-cpu ragas
%pip install -qU datasets


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [27]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("06-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
06-Evaluations


### 저장한 CSV 파일로부터 로드

- `data/ragas_synthetic_dataset.csv` 파일을 로드합니다.

In [ ]:
# If you have a CSV file, Please uncomment the following code.

# import pandas as pd
# from datasets import Dataset

# df = pd.read_csv("data/ragas_synthetic_dataset-kr.csv")

# test_dataset = Dataset.from_pandas(df)
# test_dataset

## huggingface에서 데이터 로드

huggingface에 저장한 데이터를 로드합니다.

In [28]:
# huggingface에 저장 된 데이터를 불러옵니다.
from datasets import load_dataset

test_dataset = load_dataset("haseom/ragas-test-dataset-kr", split="ragas_v1")
test_dataset

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 9
})

In [29]:
import ast

# contexts 컬럼의 문자열을 리스트로 변환
def convert_to_list(example):
    contexts = ast.literal_eval(example["contexts"])
    return {"contexts": contexts}


test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 9
})


In [30]:
test_dataset[1]["contexts"]

['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n갈릴레오의 LLM 환각 지수 평가에서 GPT-4가 가장 우수\nKEY Contents\nn 주요 LLM의 환각 현상을 평가한 ‘LLM 환각 지수’에 따르면 GPT-4는 작업 유형과 관계없이\n가장 우수한 성능을 보였으며 GPT-3.5도 거의 동등한 성능을 발휘\nn 오픈소스 모델 중에서는 메타의 라마2가 RAG 없는 질문과 답변 및 긴 형식의 텍스트\n생성에서 가장 우수한 성능을 발휘\n£주요 LLM 중 GPT-4가 가장 환각 현상 적고 GPT-3.5 터보도 비슷한 성능 기록\nn 머신러닝 데이터 관리 기업 갈릴레오(Galileo)가 2023년 11월 15일 주요 LLM의 환각 현상을 평가한\n‘LLM 환각 지수(LLM Hallucination Index)’를 발표\n∙ 생성 AI의 환각 현상은 AI 시스템이 잘못된 정보를 생성하거나, 현실과 다른 부정확한 결과를 내놓는\n현상으로, 기업의 AI 도입을 가로막는 주요 장애물이며, 환각 지수는 신뢰할 수 있는 생성 AI 구축을 위해\n환각을 평가하고 측정하는 구조화된 접근방식을 제공\n∙ 환각 지수는 △검색 증강 생성(Retrieval-Augmented Generation, RAG)*을 포함한 질문과 답변 △RAG\n없는 질문과 답변 △긴 형식의 텍스트(보고서나 기사, 에세이) 생성의 3개 작업 유형에 대하여 환각을\n기준으로 LLM의 순위를 평가\n* 기존에 학습된 데이터가 아닌 외부 소스(데이터셋, 데이터베이스, 문서 등)에서 가져온 정보를 검색해 활용하는 기술\nn 3개의 작업 유형 평가 전체에서 오픈AI의 GPT-4가 최고의 성능을 기록했으며, GPT-3.5 터보도\nGPT-4와 거의 동등한 성능을 발휘\n∙ 메타의 라마2(Llama-2-70b)는 RAG 없는 질문과 답변 유형에서 오픈소스 모델 가운데 가장 우수했고 긴\n형식의 텍스트 생성에서도 GPT-4에 준하는 성능을 기록했으나, RAG 포함 질문과 답변에서는 허깅\n페이스의 제퍼(

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import os

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("./data/AI 브리프 2023년 12월.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = AzureOpenAIEmbeddings(
  model="text-embedding-3-large",
  azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDINGS_ENDPOINT"],
  azure_deployment=os.environ["AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"],
  api_key=os.environ["AZURE_OPENAI_EMBEDDINGS_API_KEY"],
)

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = AzureChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

배치 데이터셋을 생성합니다. 배치 데이터셋은 다량의 질문을 한 번에 처리할 때 용이합니다.

- 배치: https://wikidocs.net/233345

In [32]:
batch_dataset = [question for question in test_dataset["question"]]
batch_dataset[:3]

['AI 개발과 안전의 맥락에서 AI 위험 관리의 중요성은 무엇인가요?',
 'AI 성능 평가에서 LLM 환각 지수의 의미는 무엇인가요?',
 'G7 히로시마 프로세스 첨단 AI 시스템을 위한 국제 행동 강령에 언급된 AI 위험 관리의 핵심 요소는 무엇인가요?']

`batch()` 를 호출하여 배치 데이터셋에 대한 답변을 받습니다.

In [33]:
answer = chain.batch(batch_dataset)
answer[:3]

['AI 개발과 안전의 맥락에서 AI 위험 관리의 중요성은 여러 측면에서 강조됩니다. AI 시스템의 수명주기 전반에 걸쳐 위험을 평가하고 완화하는 조치를 채택하는 것은 필수적입니다. 이는 AI 시스템의 성능과 한계를 공개하고, 적절하거나 부적절한 사용 영역을 알리는 방법으로 투명성을 보장하고 책임성을 강화하는 데 기여합니다. 또한, AI 시스템의 안전성을 보장하기 위해 국가, 국제기구, 기업, 시민사회, 학계 등 모든 이해관계자의 협력이 중요하며, 특히 첨단 AI 시스템 개발 기업은 안전 평가를 비롯한 적절한 조치를 취하여 AI 시스템의 안전을 보장할 책임이 있습니다. 이러한 위험 관리는 AI 시스템의 오용과 취약점을 파악하고 완화하는 데 도움을 주며, 사회적 위험과 안전·보안 문제를 완화하는 데 기여합니다.',
 'LLM 환각 지수는 생성 AI의 환각 현상을 평가하고 측정하는 구조화된 접근방식을 제공합니다. 환각 현상은 AI 시스템이 잘못된 정보를 생성하거나, 현실과 다른 부정확한 결과를 내놓는 현상으로, 이는 기업의 AI 도입을 가로막는 주요 장애물로 작용합니다. 환각 지수는 신뢰할 수 있는 생성 AI 구축을 위해 이러한 환각을 평가하고 측정하는 데 사용되며, 검색 증강 생성(RAG)을 포함한 질문과 답변, RAG 없는 질문과 답변, 긴 형식의 텍스트 생성의 세 가지 작업 유형에 대해 LLM의 성능을 평가합니다.',
 'G7 히로시마 프로세스 첨단 AI 시스템을 위한 국제 행동 강령에 언급된 AI 위험 관리의 핵심 요소는 다음과 같습니다:\n\n1. AI 수명주기 전반에 걸친 위험 평가와 완화.\n2. 투명성과 책임성의 보장.\n3. 정보공유와 이해관계자 간 협력.\n4. 보안 통제.\n5. 콘텐츠 인증과 출처 확인.']

LLM 이 생성한 답변을 'answer' 컬럼에 저장합니다.

In [34]:
# 'answer' 컬럼 덮어쓰기 또는 추가
if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

## 답변 평가

### Context Recall

**요약** 

- "검색된 context가 LLM 이 생성한 답변과 얼마나 일치하는지" 를 측정합니다.

Context recall은 검색된 context가 LLM 이 생성한 답변과 얼마나 일치하는지를 측정합니다. 

이는 question, ground truth 및 검색된 context를 사용하여 계산되며, 값은 0에서 1 사이로, 높을수록 더 나은 성능을 나타냅니다. 

Ground truth 답변에서 context recall을 추정하기 위해, ground truth 답변의 각 주장이 검색된 context에 귀속될 수 있는지 분석됩니다. 이상적인 시나리오에서는 ground truth 답변의 모든 주장이 검색된 context에 귀속될 수 있어야 합니다. 

$$\text{context recall} = \frac{|\text{GT claims that can be attributed to context}|}{|\text{Number of claims in GT}|}$$

### Context Precision

**요약**

- "얼마나 관련성 있는 문서가 상위에 배치되었는가?" 를 평가하는 지표입니다. 

Context Precision은 contexts 내의 ground-truth 관련 항목들이 상위 순위에 있는지를 평가하는 지표입니다. 이상적으로는 모든 관련 chunks가 상위 순위에 나타나야 합니다. 이 지표는 question, ground_truth, 그리고 contexts를 사용하여 계산되며, 0에서 1 사이의 값을 가집니다. 높은 점수일수록 더 나은 정밀도를 나타냅니다.

Context Precision@K의 계산식은 다음과 같습니다.

$$\text{Context Precision@K} = \frac{\sum_{k=1}^{K} (\text{Precision@k} \times v_k)}{\text{Total number of relevant items in the top K results}}$$

여기서 Precision@k는 다음과 같이 계산됩니다.

$$\text{Precision@k} = \frac{\text{true positives@k}}{(\text{true positives@k + false positives@k})}$$

K는 contexts의 총 chunk 수이며, $v_k \in \{0, 1\}$은 순위 k에서의 관련성 지표입니다.

이 지표는 정보 검색 시스템에서 검색된 컨텍스트의 품질을 평가하는 데 사용됩니다. 관련 정보가 얼마나 정확하게 상위 순위에 배치되었는지를 측정함으로써 시스템의 성능을 판단할 수 있습니다.

### Answer Relevancy

- "생성된 답변이 주어진 prompt에 얼마나 적절한지" 를 평가하는 지표입니다. 

이 지표의 주요 특징과 계산 방법을 요약하면 다음과 같습니다.

1. 목적: 생성된 답변의 관련성을 평가합니다.
2. 점수 해석: 낮은 점수는 불완전하거나 중복 정보를 포함한 답변을, 높은 점수는 더 나은 관련성을 나타냅니다.
3. 계산에 사용되는 요소: question, context, answer

Answer Relevancy의 계산 방법:
- 원래 question과 answer를 기반으로 생성된 합성 질문들 간의 평균 코사인 유사도로 정의됩니다.
- 수식:

$$\text{answer relevancy} = \frac{1}{N} \sum_{i=1}^N \cos(E_{g_i}, E_o)$$

또는

$$\text{answer relevancy} = \frac{1}{N} \sum_{i=1}^N \frac{E_{g_i} \cdot E_o}{\|E_{g_i}\| \|E_o\|}$$

여기서:
- $E_{g_i}$는 생성된 질문 $i$의 임베딩
- $E_o$는 원래 질문의 임베딩
- $N$은 생성된 질문의 수 (기본값 3)

주의사항:
- 실제로는 점수가 대부분 0과 1 사이에 있지만, 코사인 유사도의 특성상 수학적으로 -1에서 1 사이의 값을 가질 수 있습니다.

이 지표는 질문-답변 시스템의 성능을 평가하는 데 유용하며, 특히 생성된 답변이 원래 질문의 의도를 얼마나 잘 반영하는지를 측정합니다.

### Faithfulness

- "생성된 답변의 사실적 일관성을 주어진 컨텍스트와 비교하여 측정" 하는 지표입니다. 

주요 특징은 다음과 같습니다.

1. 목적: 답변의 사실적 일관성을 컨텍스트와 비교하여 평가합니다.
2. 계산 요소: 답변과 검색된 컨텍스트를 사용합니다.
3. 점수 범위: 0에서 1 사이로 조정되며, 높을수록 더 좋습니다.

Faithfulness 점수 계산 방법:

$$\text{Faithfulness score} = \frac{|\text{Number of claims in the generated answer that can be inferred from given context}|}{|\text{Total number of claims in the generated answer}|}$$

계산 과정:
1. 생성된 답변에서 주장(claims)들을 식별합니다.
2. 각 주장을 주어진 컨텍스트와 대조 검증하여 컨텍스트에서 추론 가능한지 확인합니다.
3. 위 수식을 사용하여 점수를 계산합니다.

예시:
- 질문: "아인슈타인은 어디서, 언제 태어났나요?"
- 컨텍스트: "알버트 아인슈타인(1879년 3월 14일 출생)은 독일 출신의 이론 물리학자로, 역사상 가장 위대하고 영향력 있는 과학자 중 한 명으로 여겨집니다."
- 높은 충실도 답변: "아인슈타인은 1879년 3월 14일 독일에서 태어났습니다."
- 낮은 충실도 답변: "아인슈타인은 1879년 3월 20일 독일에서 태어났습니다."

이 지표는 생성된 답변이 주어진 컨텍스트에 얼마나 충실한지를 평가하는 데 유용하며, 특히 질문-답변 시스템의 정확성과 신뢰성을 측정하는 데 중요합니다.

In [35]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    llm=llm,
    embeddings=embeddings,
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'context_precision': 0.6667, 'faithfulness': 0.5528, 'answer_relevancy': 0.6484, 'context_recall': 0.8333}

In [36]:
result_df = result.to_pandas()
result_df.head()

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,AI 개발과 안전의 맥락에서 AI 위험 관리의 중요성은 무엇인가요?,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 과학혁신기...,AI 개발과 안전의 맥락에서 AI 위험 관리의 중요성은 여러 측면에서 강조됩니다. ...,AI 위험 관리는 AI 기술과 관련된 잠재적 위험을 식별하고 완화하는 것을 포함하므...,1.0,0.181818,0.765443,1.0
1,AI 성능 평가에서 LLM 환각 지수의 의미는 무엇인가요?,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n갈릴레오의 LL...,LLM 환각 지수는 생성 AI의 환각 현상을 평가하고 측정하는 구조화된 접근방식을 ...,AI 성능을 평가할 때 LLM 환각 지수의 중요성은 AI가 생성한 정보의 정확성을 ...,1.0,1.000000,0.632500,1.0
2,G7 히로시마 프로세스 첨단 AI 시스템을 위한 국제 행동 강령에 언급된 AI 위험...,"[SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스를...",G7 히로시마 프로세스 첨단 AI 시스템을 위한 국제 행동 강령에 언급된 AI 위험...,G7 히로시마 프로세스 첨단 AI 시스템 국제 행동 강령에 언급된 AI 위험 관리의...,1.0,1.000000,0.800251,1.0
3,AGI(인공 일반 지능)의 주요 특징과 활용 분야는 무엇인가요?,"[구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY C...","AGI(인공 일반 지능)의 주요 특징은 성능, 범용성, 자율성입니다. AGI는 인간...","AGI(인공 일반 지능)의 주요 특징은 추론, 학습, 지각 등 인간과 유사한 인지 ...",0.0,0.478261,0.710047,0.0
4,AI 성장과 데이터 프라이버시 및 책임 있는 혁신의 관계는 무엇일까요?,"[£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\nn...",AI 성장과 데이터 프라이버시 및 책임 있는 혁신은 밀접하게 연결되어 있습니다. A...,이 문맥에서는 AI 성장과 데이터 프라이버시 사이의 연관성에 대해 논의하며 책임감 ...,1.0,0.115385,0.776504,1.0


In [37]:
result_df.to_csv("data/ragas_evaluation_result-kr.csv", index=False)

In [24]:
result_df.loc[:, "context_precision":"context_recall"]

,context_precision,faithfulness,answer_relevancy,context_recall
0,1.0,0.200000,0.765443,1.0
1,1.0,0.888889,0.622897,1.0
2,1.0,1.000000,0.800251,1.0
3,1.0,0.666667,0.661652,0.0
4,1.0,0.000000,0.813558,1.0
5,0.0,0.500000,0.739207,1.0
6,1.0,0.272727,0.680456,0.5
7,0.0,1.000000,0.000000,1.0
8,1.0,1.000000,0.644300,1.0
